In [ ]:
!pip install cloudant

In [ ]:
#!pip install scikit-learn --upgrade

In [ ]:
#!pip install lightgbm --upgrade

In [ ]:
#!pip install xgboost --upgrade

In [ ]:
#!pip install imblearn --upgrade

In [45]:
# Import de bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.impute import SimpleImputer
from cloudant import Cloudant

from sklearn.model_selection import train_test_split

# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import warnings
warnings.filterwarnings("ignore")


# Imports para formatação dos gráficos
import matplotlib as m
plt.style.use('fivethirtyeight')
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'
from matplotlib.pylab import rcParams 
rcParams['figure.figsize'] = 14,6

In [34]:
# Definicao das variaveis de conexao

user     = "ad7fd206-cbed-4b75-aa98-a9fd2e05c755-bluemix"
password = "a2537397c518fcd38947f38ce8c72c17903732bb61f838c05e5b56a29c9de0aa"
account  = "ad7fd206-cbed-4b75-aa98-a9fd2e05c755-bluemix" 
db_name  = "desafio7-iot"

In [39]:
# Funcao para carregar os dados do banco de dados NoSQL Cloudant
def obter_dados(user, password, account, db_name):

    client = Cloudant(user, password, account=account, connect=True, auto_renew=True)

    db = client[db_name]

    response = db.all_docs(limit=20000, include_docs=True)

    docs = []
    for r in response["rows"]:
        docs.append(r['doc'])

    return docs

# Verificando se existem registros duplicados no dataset
def verify_dup_rows(data):
    
    # Obtendo somente linhas duplicadas
    tmp = data[data.duplicated()]

    print("Linhas duplicadas até o momento:")
    print(tmp.shape)
    
    del tmp
    
    
# Funcao para verificar valores missing no dataset
def check_missing(df, display = 10):
    temp_df = df.copy()
    df_nan = (temp_df.isnull().sum() / len(temp_df)) * 100
    missing_data = pd.DataFrame({'Missing n': temp_df.isnull().sum(),'% Missing' :df_nan})
    if missing_data['Missing n'].sum() == 0:
        return print('Ótimo! Não há mais valores faltantes neste dataset.')
    else:
        return missing_data.sort_values('% Missing', ascending = False).head(display)

# Funcao para preencher os dados missing de algumas colunas
def preenche_missing(df):
    
    df.dropna(axis='index', how='any', subset=['Tempo', 'Estação', 'LAT', 'LONG', 
                                               'Movimentação', 'Original_473', 
                                               'Original_269', 'Zero', 'Maçã-Verde', 
                                               'Tangerina', 'Citrus', 'Açaí-Guaraná', 
                                               'Pêssego'])

    # coloca constante 0 em outras colunas com dados missing
    impute_zeros = SimpleImputer(
                                    missing_values=np.nan,
                                    strategy='constant',
                                    fill_value=0,
                                    verbose=0,
                                    copy=True
                                )
    
    impute_zeros.fit(X=df)

    # Reconstruindo um Pandas DataFrame com os resultados
    data = pd.DataFrame.from_records(
            data = impute_zeros.transform(
            X    = df
        ),
        columns=df.columns
    )    
    
    # Converte colunas para numerico
    data = data.apply(pd.to_numeric, errors='ignore')

    return data

# Funcao para trabalhar as features
def fe(df):

    # Removendo colunas unicas
    data = df.drop(['Tempo', 'Estação', 'LAT', 'LONG', 'Movimentação', '_id', '_rev', 'row'], axis = 1)

    # Removendo registros duplicados
    data = data.drop_duplicates().reset_index(drop=True)

    return data

# Funcao para transformar features categoricas em numericas
def fe_cat_num(df):

    # Tratando variáveis categóricas com o método Pandas ``get_dummies()''
    df = pd.get_dummies(df, columns=['xxxx'])

    return df    

In [40]:
# Criando um dataframe a partir da lista de dados vindo do IoT e gravados no NoSQL
train = pd.DataFrame(data=obter_dados(user, password, account, db_name))

# Carregando o dataset de testes 
test  = pd.read_csv(r'teste_desafio_7.csv')


In [41]:
# Feature Engineering
treino = preenche_missing(train)    
treino = fe(treino)   

# Removendo outliers
#treino = treino[treino['modulos_finalizados'] <= 500]

# Verificando as colunas com dados missing do dataset
check_missing(treino, display = 15)

print(treino.shape)

print('\n')
# Verificar os registros duplicados
verify_dup_rows(treino)

treino.head(10)

Ótimo! Não há mais valores faltantes neste dataset.
(3482, 9)


Linhas duplicadas até o momento:
(0, 9)


,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET
0,45,18,59,14,32,19,13,21,NORMAL
1,65,31,65,23,20,37,15,15,NORMAL
2,62,55,36,43,35,25,24,26,NORMAL
3,11,43,65,39,43,33,3,35,REABASTECER
4,59,65,40,20,39,9,43,34,NORMAL
5,63,65,65,31,19,20,13,43,NORMAL
6,86,49,34,14,17,43,22,35,NORMAL
7,17,52,59,40,7,28,3,29,REABASTECER
8,76,65,44,24,15,39,13,21,NORMAL
9,86,27,24,2,37,3,36,8,REABASTECER


In [42]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3482 entries, 0 to 3481
Data columns (total 9 columns):
Original_473    3482 non-null int64
Original_269    3482 non-null int64
Zero            3482 non-null int64
Maçã-Verde      3482 non-null int64
Tangerina       3482 non-null int64
Citrus          3482 non-null int64
Açaí-Guaraná    3482 non-null int64
Pêssego         3482 non-null int64
TARGET          3482 non-null object
dtypes: int64(8), object(1)
memory usage: 245.0+ KB


In [53]:
# Selecionando features para treinamento
X = treino[['Original_473', 'Original_269', 'Zero', 'Maçã-Verde', 'Tangerina', 'Citrus', 'Açaí-Guaraná', 'Pêssego']]

# Selecionando feature target
y = treino['TARGET']

In [54]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_gbc = GradientBoostingClassifier().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_gbc.predict(X_valid)

# Salvando o score
gbc_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.78      0.84      0.81       470
 REABASTECER       0.61      0.51      0.55       227

    accuracy                           0.73       697
   macro avg       0.70      0.68      0.68       697
weighted avg       0.73      0.73      0.73       697

F1_macro   :  0.6826242662170644
F1_micro   :  0.7345767575322812
F1_weighted:  0.7273918385206022
F1_none    :  [0.81103166 0.55421687]


In [55]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_lr = LogisticRegression().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_lr.predict(X_valid)

# Salvando o score
lr_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.69      0.94      0.80       470
 REABASTECER       0.51      0.12      0.19       227

    accuracy                           0.68       697
   macro avg       0.60      0.53      0.49       697
weighted avg       0.63      0.68      0.60       697

F1_macro   :  0.4949923057194152
F1_micro   :  0.6757532281205165
F1_weighted:  0.6003278072625029
F1_none    :  [0.79712747 0.19285714]


In [56]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_knn = KNeighborsClassifier().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_knn.predict(X_valid)

# Salvando o score
knn_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.72      0.90      0.80       470
 REABASTECER       0.57      0.26      0.36       227

    accuracy                           0.70       697
   macro avg       0.64      0.58      0.58       697
weighted avg       0.67      0.70      0.66       697

F1_macro   :  0.5809112154834025
F1_micro   :  0.6958393113342898
F1_weighted:  0.6574249888855002
F1_none    :  [0.80037665 0.36144578]


In [57]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_dtc = DecisionTreeClassifier().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_dtc.predict(X_valid)

# Salvando o score
dtc_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.80      0.83      0.81       470
 REABASTECER       0.61      0.57      0.59       227

    accuracy                           0.74       697
   macro avg       0.71      0.70      0.70       697
weighted avg       0.74      0.74      0.74       697

F1_macro   :  0.702410285155458
F1_micro   :  0.7431850789096126
F1_weighted:  0.7408144513657664
F1_none    :  [0.81256545 0.59225513]


In [58]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_rfc = RandomForestClassifier().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_rfc.predict(X_valid)

# Salvando o score
rfc_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.87      0.79      0.83       470
 REABASTECER       0.64      0.76      0.69       227

    accuracy                           0.78       697
   macro avg       0.76      0.78      0.76       697
weighted avg       0.80      0.78      0.79       697

F1_macro   :  0.7625681296615032
F1_micro   :  0.7819225251076041
F1_weighted:  0.7862018889248324
F1_none    :  [0.83035714 0.69477912]


In [59]:
# Split dos dados em treino e validacao
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=133)

# Treinamento com GradientBoosting
model_xgb = XGBClassifier().fit(X_train, y_train)

# Validacao do modelo
y_pred_val = model_xgb.predict(X_valid)

# Salvando o score
xgb_score = f1_score(y_valid, y_pred_val, average='weighted')

# Matriz de Classificacao
print(classification_report(y_valid, y_pred_val))

# Avaliacao da metrica F1
print('F1_macro   : ', f1_score(y_valid, y_pred_val, average='macro'))
print('F1_micro   : ', f1_score(y_valid, y_pred_val, average='micro'))
print('F1_weighted: ', f1_score(y_valid, y_pred_val, average='weighted'))
print('F1_none    : ', f1_score(y_valid, y_pred_val, average=None))

              precision    recall  f1-score   support

      NORMAL       0.83      0.81      0.82       470
 REABASTECER       0.62      0.66      0.64       227

    accuracy                           0.76       697
   macro avg       0.73      0.73      0.73       697
weighted avg       0.76      0.76      0.76       697

F1_macro   :  0.7279035547886508
F1_micro   :  0.757532281205165
F1_weighted:  0.7592069483504463
F1_none    :  [0.81769148 0.63811563]


In [60]:
# Prepara a lista de resultados
metricas = [(gbc_score),(lr_score),(knn_score),(dtc_score),(rfc_score),(xgb_score)]

# Cria o dataframe
df_metricas = pd.DataFrame(metricas, 
                           columns = ['Accuracy'], 
                           index = ['GBC', 'LR', 'KNN', 'DTC', 'RFC', 'XGB']) 

# Visualiza o resultado
df_metricas

,Accuracy
GBC,0.727392
LR,0.600328
KNN,0.657425
DTC,0.740814
RFC,0.786202
XGB,0.759207
